In [1]:
#ProgramName: Daily-U-UpdateRowCountDifferenceValues-DEV
#Purpose: Update default values(zero) differences with calculated changes captured
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the metadata captured

import os, sys, argparse, csv, pyodbc, sql, time, datetime
import sqlalchemy as db

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [2]:
################################################################################
# Variables: Constructing environment variables for automation steps 
################################################################################
now01 = datetime.datetime.now()
csvtype = 'Base' # csvtype - use 'Test or Base'
Dbug = 'Off'     # Debugging - use On or Off  
# Process files in procpath
pfile00 = 'dbList.csv'
pfile01 = 'tblList.csv'
pfile02 = 'schemaList.csv'
# Used for debugging output
file00 = 'DiffReport0.csv' 
file01 = 'DiffReport1.csv' #ApplicationData - used for new tables found (begin plots)
file02 = 'DiffReport2.csv' #Assessment - used for new tables found (begin plots)
file03 = 'DiffReport3.csv' #Control - used for new tables found (begin plots)
file04 = 'DiffReport4.csv' #Downstreamfeeds - used for new tables found (begin plots)
file05 = 'DiffReport5.csv' #DownstreamFeedsV15 - used for new tables found (begin plots)
file06 = 'DiffReport6.csv' #GCPS_Operations - used for new tables found (begin plots)
file07 = 'DiffReport7.csv' #GSDR - used for new tables found (begin plots)
file08 = 'DiffReport8.csv' #GSDR_Synergy - used for new tables found (begin plots)
file09 = 'DiffReport9.csv' #GSDR_Synergy_Temp - used for new tables found (begin plots)
file10 = 'DiffReport10.csv'#GSDR_Temp - used for new tables found (begin plots)
file11 = 'DiffReport11.csv'#ODS_WebApps - used for new tables found (begin plots)
file12 = 'DiffReport12.csv'#PPROD  - used for new tables found (begin plots)
file13 = 'DiffReport13.csv'#Predictive_Analytics - used for new tables found (begin plots)
file14 = 'DiffReport14.csv'
file15 = 'DiffReport15.csv'



In [3]:
################################################################################
# Specify pathariables: Make sure output location is available for write: ex. UNIX touch 
################################################################################
now01 = datetime.datetime.now()
import os, errno, pathlib2
try:
    os.makedirs(r"C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Process", exist_ok=True)  # succeeds even if directory exists.
    os.makedirs(r"C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Reports", exist_ok=True)
    os.makedirs(r"C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\config", exist_ok=True)
except FileExistsError:
    # directory already exists
    pass

procpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Process')
csvpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Reports')
sqlpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql')
configpath = (r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\config')

krange = (0,1,2,3,4,5,6,7,8,9)
for k in krange:
    with open(os.path.join(sqlpath, 'sqlfile0{0}.sql').format(k), 'w') as fp: pass
    with open(os.path.join(sqlpath, 'sqlfile1{0}.sql').format(k), 'w') as fp: pass
    with open(os.path.join(csvpath, 'DiffReport0{0}.csv').format(k), 'w') as fp: pass
    with open(os.path.join(csvpath, 'DiffReport1{0}.csv').format(k), 'w') as fp: pass
now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-05 07:15:03


In [4]:
#Read excel file with core source tables
dbList = pd.read_csv(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Process\dbList.csv', encoding='1252')
tblList = pd.read_csv(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Process\tblList.csv', encoding='1252')
schemaList = pd.read_csv(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\Process\schemaList.csv', encoding='1252')

In [5]:
dbList.head()

,Greg


In [6]:
tblList.head()

,tableRowCountApplicationData
0,tableRowCountAssessment
1,tableRowCountControl
2,tableRowCountDownstreamFeeds
3,tableRowCountDownstreamFeedsV15
4,tableRowCountGCPS_Operations


In [7]:
schemaList.head()

,dbo


In [8]:
conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
sql_query = pd.read_sql_query(''' 
SELECT DISTINCT [TableName] FROM [Greg].[dbo].[tableRowCountApplicationData] ORDER BY 1;
'''
                              ,conn)
conn.close()

In [9]:
sql_query.head()

,TableName
0,AcademicContract
1,AcademicContractIntervention
2,AccessRights
3,AccommodationToColMap
4,Addendum_Log


#Manually update existing tables and diff data values - automate to capture new tables IRT (in real time pending) 

In [10]:
#Read into the df the sql file built as a CTE update command
        
s01 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_appdata_set_difference.sql')
sqlFile01 = s01.read()
s01.close()

for sqlCmd in sqlFile01.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:15:23


In [11]:
s02 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_assessment_set_difference.sql')
sqlFile02 = s02.read()
s02.close()

for sqlCmd in sqlFile02.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


Difference Value updated - Process Complete:  2021-05-05 07:17:45


In [12]:
s03 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_control_set_difference.sql')
sqlFile03 = s03.read()
s03.close()

for sqlCmd in sqlFile03.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


Difference Value updated - Process Complete:  2021-05-05 07:17:54


In [13]:
s04 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_downstreamfeeds_set_difference.sql')
sqlFile04 = s04.read()
s04.close()

for sqlCmd in sqlFile04.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:20:40


In [14]:
s05 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_downstreamfeedsV15_set_difference.sql')
sqlFile05 = s05.read()
s05.close()

for sqlCmd in sqlFile05.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:20:44


In [15]:
s06 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_gcps_operations_set_difference.sql')
sqlFile06 = s06.read()
s06.close()

for sqlCmd in sqlFile06.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:20:50


In [16]:
s07 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_gsdr_set_difference.sql')
sqlFile07 = s07.read()
s07.close()

for sqlCmd in sqlFile07.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:24:30


In [17]:
s08 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_gsdr_synergy_set_difference.sql')
sqlFile08 = s08.read()
s08.close()

for sqlCmd in sqlFile08.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:24:34


In [18]:
s09 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_gsdr_synergy_temp_set_difference.sql')
sqlFile09 = s09.read()
s09.close()

for sqlCmd in sqlFile09.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:24:38


In [19]:
s10 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_gsdr_temp_set_difference.sql')
sqlFile10 = s10.read()
s10.close()

for sqlCmd in sqlFile10.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:28:18


In [20]:
s11 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_ods_webapps_set_difference.sql')
sqlFile11 = s11.read()
s11.close()

for sqlCmd in sqlFile11.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:28:23


In [21]:
s12 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_pprod_set_difference.sql')
sqlFile12 = s12.read()
s12.close()

for sqlCmd in sqlFile12.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

Difference Value updated - Process Complete:  2021-05-05 07:28:31


In [22]:
s13 = open(r'C:\Users\e201873842\Documents\Jupyter\Daily\Difference\sql\update_predictive_analytics_set_difference.sql')
sqlFile13 = s13.read()
s13.close()

for sqlCmd in sqlFile13.split(';'):
    try:
        conn = pyodbc.connect('Server=DEVODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Greg')
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
    except:
        continue
    finally:
        conn.commit()
        continue

conn.commit()
conn.close()

now01 = datetime.datetime.now()
print('Difference Value updated - Process Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))


Difference Value updated - Process Complete:  2021-05-05 07:29:16


In [23]:
print('Process Complete: All diff values updated: Next validate and run new plots', now01.strftime("%Y-%m-%d %H:%M:%S"))

Process Complete: All diff values updated: Next validate and run new plots 2021-05-05 07:29:16
